In [1]:
import findspark
findspark.init('/home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark')

In [2]:
import astroprov
import numpy as np
import os

from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column

import collections
import subprocess

import time
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool

from functools import partial

from astropy.io import ascii
#from sqlalchemy import create_engine
import sqlite3
import mysql.connector
from pandas import DataFrame

import mysql.connector
from mysql.connector import Error
from mysql.connector import pooling

from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import SparkConf
import datetime
from sqlalchemy import create_engine


In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark/mysql-connector-java_8.0.16-1ubuntu16.04_all/usr/share/java/mysql-connector-java-8.0.16.jar  pyspark-shell'
config = SparkConf().setAll([('spark.executor.cores', '6'),('spark.cores.max', '6'),('spark.driver.memory','1g'),('spark.executor.memory', '500m')])


In [4]:
sc = SparkContext(appName='App',conf=config)
sqlContext = SQLContext(sc)
dataframe_mysql = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://localhost/Kepler").option("driver", "com.mysql.jdbc.Driver").option("dbtable", "starlist").option("user", "mj1e16").option("password", "[sqlT1G3R]").load()
dataframe_mysql.registerTempTable('starlist')

In [66]:
def innerJoin(tableName,provDir,totGross,diffSize=1,xlength=1015.,ylength=1085.):
    nsegs = 4
    
    temptableName = tableName[0:5]+tableName[-1]
    df = sqlContext.sql("SELECT NUMBER, MAG_BEST, X_IMAGE, Y_IMAGE, ELLIPTICITY FROM {} INNER JOIN starlist ON {}.X_IMAGE BETWEEN starlist.X_POS_MIN AND starlist.X_POS_MAX AND {}.Y_IMAGE BETWEEN starlist.Y_POS_MIN AND starlist.Y_POS_MAX".format(temptableName,temptableName,temptableName))
    df.registerTempTable(temptableName+'match')
    sqlContext.cacheTable(temptableName+'match')
    totNumber = df.count()
    print('tot',totNumber)
    
    xsegment = xlength/nsegs
    ysegment = ylength/nsegs
    
    variableList = [tableName]
    #df.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='result_{}'.format(tableName),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    for xsegs in range(nsegs):
        xmin = int(xsegs*xsegment)+15
        xmax = int(xmin+xsegment)+15
        for ysegs in range(nsegs):
            ymin = int(ysegs*ysegment)+15
            ymax = int(ymin+ysegment)+15
            
            df = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(temptableName+'match',xmin,xmax,ymin,ymax))
            
            number = df.count()
            print('x {} y {} num {}'.format(xsegs,ysegs,number))
            variableList.append(number)
            # need to create some place to write to
    magRange = np.linspace(15,25,11)
    #magRange = [] # update with observed mag range from sextractor
    for mag in range(len(magRange)):
        if magRange[mag] != magRange[-1]:
            magMin = magRange[mag]
            magMax = magRange[mag+1]
            df = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.MAG_BEST BETWEEN {1} AND {2}".format(temptableName+'match',magMin,magMax))
            number = df.count()
            print('mag {} number {}'.format(mag,number))
            variableList.append(number)
            # record results somewhere
    
    
    
    astroprov.provcall([tableName,'starlist'],['result_{}'.format(tableName)],"innerJoin_Python2Python_SQ_tmpl.provn","innerJoin",provDir)
    
    sqlContext.uncacheTable(temptableName+'match')
    sqlContext.dropTempTable(temptableName+'match')
    sqlContext.uncacheTable(temptableName)
    sqlContext.dropTempTable(temptableName)
    variableList.append(totNumber)
    variableList.append(totGross)
    variableTuple = tuple(variableList)
    print(variableTuple)
    return variableTuple

def makeConfig(valList,provDir,tableName='table',defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',attributeList=['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']):
    
    #workAroundList = [16,32,64,128,256,512]
    workAroundList = [1,3,5,7,9,11]
    ident = workAroundList.index(valList[-1])
    tableName = tableName[ident] # could just return a letter?
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    
    catLocFinder = 'CATALOG_NAME'
    catname = 'test'+str(ident)+'.cat'
    nameLoc = data.find(catLocFinder) + len(catLocFinder)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' '+catname+' ' + data[endLoc:]
    data = newData 
        
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData    
    
    
    confName = 'default_'+str(ident)+'.sex'
    with open(defaultDir+confName,'w') as f:
        f.write(data)
    astroprov.provcall(valList,[confName],"makeConfig_Python2Python_SQ_tmpl.provn","makeConfig",provDir)
    return(confName,tableName,catname)



def findObjects(confName,tableName,catName,provDir,defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',imagename='/home/mj1e16/iraf/editedImage5000.fits'):
    
    astroprov.provcall([confName,'/home/mj1e16/iraf/editedImage5000.fits'],[tableName],"findObjects_Python2DaoStarfidner_SQ_tmpl.provn","findObjects",provDir)

    os.chdir(defaultDir)
    subprocess.call(['sex',imagename,'-c',confName])
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    df_spark = sqlContext.createDataFrame(df2)
    temptableName = tableName[0:5]+tableName[-1]
    #print(temptableName)
    df_spark.registerTempTable(temptableName)
    totGross = df_spark.count()
    print('Gross ',totGross)
    sqlContext.cacheTable(temptableName)
    # count rows in table and add to some big table
    
    #df_spark.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='{}'.format(tableName),user='mj1e16',password='[sqlT1G3R]').mode('append').save()

    return totGross

def alltogethernow(valList,tableName,IMAGE,provDir):
    Names = makeConfig(valList,provDir,tableName=tableName)
    totGross = findObjects(Names[0],Names[1],Names[2],provDir,imagename=IMAGE) # confName tabName catname
    finalTuple = innerJoin(Names[1],provDir,totGross)
    return finalTuple

In [62]:
valList = [np.linspace(1,10,10),np.linspace(1,9,9),np.linspace(8,128,16),[1,3,5,7,9,11]] # astro start

In [63]:
def evaluateImage(valList,simImage,provDir,smallName='Poshak',atList=['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']):
    finalTableTuples = []
    p = ThreadPool(6)
    t0 = time.time()
    for x0 in range(1): #len(valList[0])):
        for x1 in range(1): #len(valList[1])):
                for x2 in range(1): #len(valList[2])):
                    name = smallName+'_'+str(x0)+'_'+str(x1)+'_'+str(x2)+'_'
                    fullname = [name]*len(valList[3])
                    fullValList = []
                    for x3 in range(len(valList[3])):
                        fullname[x3] += str(x3)
                        print(fullname[x3])
                        fullValList.append([valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3]])

                        #alltogethernow(fullValList[x4],fullname,imageName,provDir)
                    try:
                        finalTableTuples.extend(p.map(partial(alltogethernow,tableName=fullname,IMAGE=simImage,provDir=provDir),fullValList))
                    except RuntimeError:
                        with open('redos','a') as f:
                            bigString = simImage+','+str(fullname)+','+str(fullValList)
                            f.write(bigString)
                        print('Redo with valList = ',fullValList)
                        pass

    print(finalTableTuples)
    #def alltogethernow(valList,tableName,IMAGE,provDir):
    rdd = sc.parallelize(finalTableTuples)
    kepler = rdd.map(lambda x: Row(tabName=x[0], xy_0=int(x[1]),xy_1=int(x[2]),xy_2=int(x[3]),xy_3=int(x[4]),xy_4=int(x[5]),xy_5=int(x[6]),xy_6=int(x[7]),xy_7=int(x[8]),xy_8=int(x[9]),xy_9=int(x[10]),xy_10=int(x[11]),xy_11=int(x[12]),xy_12=int(x[13]),xy_13=int(x[14]),xy_14=int(x[15]),xy_15=int(x[16]),mag_0=int(x[17]),mag_1=int(x[18]),mag_2=int(x[19]),mag_3=int(x[20]),mag_4=int(x[21]),mag_5=int(x[22]),mag_6=int(x[23]),mag_7=int(x[24]),mag_8=int(x[25]),mag_9=int(x[26]),totNum=int(x[27]),totGross=int(x[28])))
    schemaKepler = sqlContext.createDataFrame(kepler)
    schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='finalResults',user='mj1e16',password='[sqlT1G3R]').mode('append').save()

    print(time.time()-t0)

In [64]:
sqlContext.cacheTable('starlist')

In [67]:
### Make function to replace starlist
evaluateImage(valList,'/home/mj1e16/Simages/diff_44_1_alt.fits','/home/mj1e16/keplerPhotometry/provDump/44/selectiveMedian/',smallName='Poshak_44_1')

Poshak_44_1_0_0_0_0
Poshak_44_1_0_0_0_1
Poshak_44_1_0_0_0_2
Poshak_44_1_0_0_0_3
Poshak_44_1_0_0_0_4
Poshak_44_1_0_0_0_5
('Gross ', 16199)
('Gross ', 16284)
('Gross ', 16185)
('Gross ', 16471)
('Gross ', 16123)
('tot', 303)
('Gross ', 16024)
x 0 y 0 num 62
('tot', 307)
('tot', 306)
x 0 y 1 num 13
('tot', 305)
('tot', 303)
x 0 y 0 num 60
x 0 y 0 num 62
x 0 y 2 num 11
x 0 y 0 num 59
x 0 y 0 num 66
x 0 y 1 num 13
('tot', 307)
x 0 y 1 num 13
x 0 y 3 num 7
x 0 y 1 num 14
x 0 y 1 num 14
x 0 y 2 num 13
x 0 y 0 num 61
x 0 y 2 num 12
x 0 y 2 num 10
x 0 y 3 num 8
x 1 y 0 num 4
x 0 y 2 num 11
x 0 y 1 num 13
x 0 y 3 num 8
x 0 y 3 num 9
x 1 y 1 num 17
x 1 y 0 num 4
x 0 y 3 num 8
x 0 y 2 num 12
x 1 y 0 num 5
x 1 y 2 num 29 x 1 y 1 num 17

x 1 y 0 num 4
x 0 y 3 num 8
x 1 y 0 num 4
x 1 y 1 num 17
x 1 y 2 num 30
x 1 y 1 num 17
x 1 y 0 num 5
x 1 y 3 num 9
x 1 y 2 num 29 x 1 y 1 num 17

x 1 y 3 num 9
x 1 y 2 num 30
x 1 y 1 num 16
x 1 y 3 num 9
x 2 y 0 num 10
x 1 y 2 num 30
x 2 y 0 num 10
x 1 y 2 num 29
x 

In [15]:
def test():
    row = [('tabName',1,1),('tabName2',1,2)]
    rdd = sc.parallelize(row)
    kepler = rdd.map(lambda x: Row(tabName=x[0], xyRegion=int(x[1]),magRegion=int(x[2])))
    schemaKepler = sqlContext.createDataFrame(kepler)
    df = schemaKepler.toPandas()
    print(df)

In [24]:
def testPartial(value):
    return (value,value)

In [25]:
p = ThreadPool(6)
values = [0,1,2,3,4,5]
answer = []
for x in range(5):
    answer.extend(p.map(testPartial,values))
print(answer)

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]


In [26]:
import sys

In [27]:
sys.getsizeof(answer)

336

In [40]:
def findMagRange(catName):
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    #df3 = df[df['MAG_BEST'] == 99.0000]
#     maxVal = df2['MAG_BEST'].max()
#     minVal = df2['MAG_BEST'].min()
    df4 = df2.sort_values('MAG_BEST')
    return df4
    #print(minVal,maxVal)

In [37]:
for x in range(5):
    print('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))
    findMagRange('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))

/home/mj1e16/sextractor/sextractor-master/config/test0.cat
(15.5602, 28.113700000000001)
/home/mj1e16/sextractor/sextractor-master/config/test1.cat
(15.3682, 29.890799999999999)
/home/mj1e16/sextractor/sextractor-master/config/test2.cat
(15.274900000000001, 28.160399999999999)
/home/mj1e16/sextractor/sextractor-master/config/test3.cat
(15.155099999999999, 28.525600000000001)
/home/mj1e16/sextractor/sextractor-master/config/test4.cat
(14.946, 26.240400000000001)


In [45]:
x = 2
df = findMagRange('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))

In [46]:
df

,NUMBER,EXT_NUMBER,FLUX_ISO,FLUXERR_ISO,MAG_BEST,BACKGROUND,THRESHOLD,FLUX_MAX,XPEAK_IMAGE,YPEAK_IMAGE,X_IMAGE,Y_IMAGE,FWHM_IMAGE,ELLIPTICITY
14174,14175,1,7398.887000,47.335390,15.2749,1.999571,2.000285,228.491500,214,914,210.8495,914.5341,37.06,0.132
2854,2855,1,7306.028000,38.005420,15.3114,4.175594,2.000285,340.770800,315,97,312.9309,94.1305,22.04,0.626
4358,4359,1,6832.718000,48.131700,15.3523,4.736762,2.000285,201.457700,332,75,327.9319,90.9586,49.65,0.736
1880,1881,1,7123.002000,18.976370,15.3624,6.686461,2.000285,998.966000,321,88,320.9205,85.6949,9.96,0.622
14821,14822,1,6764.762000,43.318990,15.3782,1.787364,2.000285,769.138400,578,906,576.8671,906.1858,31.85,0.347
5383,5384,1,6076.962000,55.613690,15.4282,3.390419,2.000285,112.598000,727,200,745.1295,197.9760,73.44,0.155
6942,6943,1,6501.120000,27.644490,15.4521,2.470802,2.000285,494.882300,168,336,168.7372,335.8751,19.36,0.695
2110,2111,1,5578.957000,40.005700,15.5852,2.044692,2.000285,360.920000,39,73,36.3859,74.4796,15.04,0.197
1407,1408,1,7263.567000,43.549290,15.6573,-0.061376,2.000285,779.172700,258,42,254.5778,43.2272,13.74,0.168
3901,3902,1,4530.057000,19.598710,15.8492,4.250020,2.000285,501.520900,732,204,731.5367,203.5862,10.07,0.561


In [49]:
magRange = np.linspace(15,30,16)

In [50]:
magRange

array([ 15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,
        26.,  27.,  28.,  29.,  30.])